# kernel

> IPythonKernel based on pystata

The latest documentation for implementing a wrapper kernel is [here](https://jupyter-client.readthedocs.io/en/latest/wrapperkernels.html), but the current code deviates from those instructions, instead inheriting from the IPython kernel implementation, [IPythonKernel](https://github.com/ipython/ipykernel/blob/main/ipykernel/ipkernel.py).

In [ ]:
#| default_exp kernel
%load_ext autoreload
%autoreload 2

In [ ]:
#| hide
from nbdev.showdoc import *
from fastcore.test import test_eq

In [ ]:
#| export
from nbstata.config import get_config, launch_stata
from nbstata.magics import StataMagics
from fastcore.basics import patch_to
from ipykernel.ipkernel import IPythonKernel
import os
import sys
from packaging import version

In [ ]:
#| export
class PyStataKernel(IPythonKernel):
    implementation = 'nbstata'
    implementation_version = '0.0.1'
    language = 'stata'
    language_version = '17'
    language_info = {
        'name': 'stata',
        'mimetype': 'text/x-stata',
        'codemirror_mode': 'stata',
        'file_extension': '.do',
    }
    banner = "nbstata: a Jupyter kernel for Stata based on pystata"

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.stata_ready = False
        self.shell.execution_count = 0
        self.magic_handler = None
        self.env = None

In [ ]:
#| export
@patch_to(PyStataKernel)
def init_stata(self):
    def _set_graph_format(graph_format):
        if graph_format == 'nbstata':
            pass
        else:
            from pystata.config import set_graph_format
            set_graph_format(graph_format)
    
    self.env = get_config()
    if self.env['echo'] not in ('True', 'False', 'None'):
        raise OSError("'" + self.env['echo'] + "' is not an acceptable value for 'echo'.")

    launch_stata(self.env['stata_dir'], self.env['edition'],
                 False if self.env['splash']=='False' else True)

    _set_graph_format(self.env['graph_format'])

    self.magic_handler = StataMagics()

    self.stata_ready = True

In [ ]:
#| export
class Cell:
    def __init__(self, kernel, code_w_magics):
        if kernel.env['echo'] == 'None':
            self.noecho = True
            self.echo = False
        elif kernel.env['echo'] == 'True':
            self.noecho = False
            self.echo = True
        else:
            self.noecho = False
            self.echo = False
        self.quietly = False
        self.code = kernel.magic_handler.magic(code_w_magics, kernel, self)
    
    def run(self):
        if self.code != '':
            if self.noecho and not self.quietly:
                from nbstata.helpers import run_noecho
                run_noecho(self.code)
            else:
                from pystata.stata import run
                run(self.code, quietly=self.quietly, inline=True, echo=self.echo)

Some `Cell` functionality can be tested apart from a kernel:

In [ ]:
from unittest.mock import Mock

In [ ]:
kernel1 = Mock()
kernel1.env = {'echo': 'None'}
kernel1.magic_handler = StataMagics()

code_w_magics = '''disp "test output"'''
cell1 = Cell(kernel1, code_w_magics)
cell1.code

'disp "test output"'

In [ ]:
#| eval: false
launch_stata(splash=False)
cell1.run()

test output


In [ ]:
code_w_magics = '''*%quietly
disp "test output"'''
cell2 = Cell(kernel1, code_w_magics)
test_eq(cell2.quietly, True)

In [ ]:
#| eval: false
cell2.run()

In [ ]:
kernel1.env = {'echo': 'True'}

code_w_magics = '''disp "test output"'''
cell3 = Cell(kernel1, code_w_magics)
test_eq(cell3.noecho, False)

In [ ]:
#| eval: false
cell3.run()

. disp "test output"
test output


In [ ]:
code_w_magics = '''*%noecho
disp "test output"'''
cell4 = Cell(kernel1, code_w_magics)
test_eq(cell4.noecho, True)

In [ ]:
#| eval: false
cell4.run()

test output


In [ ]:
#| export
def print_red(text):
    print(f"\x1b[31m{text}\x1b[0m")

In [ ]:
print_red("test_red")

test_red


In [ ]:
#| export
@patch_to(PyStataKernel)
def do_execute(self, code, silent, store_history=True, user_expressions=None,
               allow_stdin=False):
    if not self.stata_ready:
        self.init_stata()
    self.shell.execution_count += 1
    try:
        Cell(self, code).run()
        return {
            'status': 'ok',
            'execution_count': self.execution_count,
            'payload': [],
            'user_expressions': {},
            }
    except SystemError as err:
        reply_content = {
            "traceback": [],
            "ename": "Stata error",
            "evalue": str(err),
        }
        if not silent:
            print_red(reply_content['evalue'])
#             self.send_response(
#                 self.iopub_socket,
#                 "error",
#                 reply_content,
#             )
        reply_content.update({
            'status': "error",
            'execution_count': self.execution_count,
        })
        return reply_content

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()